In [1]:
from scipy.signal import butter, lfilter
import scipy
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as Data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
import time
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load dataset

In [2]:
dataset_path = "../data/physionet_processed/"

In [3]:
dataset_1=np.load(dataset_path + '1.npy')
print('The shape of Dataset_1:', dataset_1.shape)
dataset_1

The shape of Dataset_1: (259520, 65)


array([[-16, -29,   2, ..., -11,  15,   0],
       [-56, -54, -27, ...,   1,  21,   0],
       [-55, -55, -29, ...,  18,  35,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9],
       [  0,   0,   0, ...,   0,   0,   9]], dtype=int64)

# Filtering

In [4]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = scipy.signal.lfilter(b, a, data)
    return y

In [5]:
n_fea = 64  # 64 channels
label = dataset_1[:, n_fea: n_fea+1]  # seperate label from feature
feature = dataset_1[:, 0:n_fea]
feature_f=[]  # feature after filtering

# EEG Delta pattern decomposition
for i in range(feature.shape[1]):
    x = feature[:, i]
    fs = 160.0
    lowcut = 0.5
    highcut = 4.0
    y = butter_bandpass_filter(x, lowcut, highcut, fs, order=3)
    feature_f.append(y)

feature_f=np.array(feature_f).T
print('The shape of filtered feature:',feature_f.shape)

data_f=np.hstack((feature_f,label))  # stack label to filtered feature
print("The shape of dataset_1 after filtering:",data_f.shape)

The shape of filtered feature: (259520, 64)
The shape of dataset_1 after filtering: (259520, 65)


In [6]:
def extract(input, n_classes, n_fea, time_window, moving):
    xx = input[:, :n_fea]
    yy = input[:, n_fea:n_fea + 1]
    new_x = []
    new_y = []
    number = int((xx.shape[0] / moving) - 1)
    for i in range(number):
        ave_y = np.average(yy[int(i * moving):int(i * moving + time_window)])
        if ave_y in range(n_classes + 1):
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(ave_y)
        else:
            new_x.append(xx[int(i * moving):int(i * moving + time_window), :])
            new_y.append(0)

    new_x = np.array(new_x)
    new_x = new_x.reshape([-1, n_fea * time_window])
    new_y = np.array(new_y)
    new_y.shape = [new_y.shape[0], 1]
    data = np.hstack((new_x, new_y))
    data = np.vstack((data, data[-1]))  # add the last sample again, to make the sample number round
    return data

In [7]:
n_class = 10  # 0~9 classes ('10:rest' is not considered)
segment_length = 16  # selected time window; 16=160*0.1

# segment data, check more details about the 'extract' function in BCI_functions.ipynb
data_seg = extract(dataset_1, n_classes=n_class, n_fea=n_fea, time_window=segment_length, moving=(segment_length/2))  # 50% overlapping

print('After segmentation, the shape of the data:', data_seg.shape)

After segmentation, the shape of the data: (32440, 1025)


In [8]:
# remove instance with label==10 (rest)
# removed_label = [2,3,4,5,6,7,8,9,10]  #2,3,4,5,
removed_label = []
for ll in removed_label:
    id = dataset_1[:, -1]!=ll
    dataset_1 = dataset_1[id]

# data segmentation
n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 3  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 40

data_seg = extract(dataset_1, n_classes=n_class, n_fea=no_feature, time_window=segment_length, moving=(segment_length/2))  # 50% overlapping
print('After segmentation, the shape of the data:', data_seg.shape)


After segmentation, the shape of the data: (173013, 193)


In [9]:
# split training and test data
no_longfeature = no_feature*segment_length
data_seg_feature = data_seg[:, :no_longfeature]
data_seg_label = data_seg[:, no_longfeature:no_longfeature+1]
train_feature, test_feature, train_label, test_label = train_test_split(data_seg_feature, data_seg_label, shuffle=True)

In [10]:
train_feature

array([[-17.,  -4.,  13., ...,  55.,  83.,  23.],
       [ 48.,  52.,  73., ...,  37.,  12.,  20.],
       [ 25.,  28.,  15., ..., -26.,  -5., -51.],
       ...,
       [-49., -26., -21., ..., -28.,   6., -23.],
       [ 29.,  24.,  29., ...,  25.,  -8.,  56.],
       [-34., -39., -32., ..., -52., -51., -46.]])

In [11]:
test_feature

array([[ 79.,  67.,  43., ..., -15.,   3., -11.],
       [ 91.,  69.,  30., ...,  19.,  43.,  32.],
       [  5.,   6.,  -8., ..., -16.,  36., -12.],
       ...,
       [-66., -50., -69., ..., -29.,  -3.,  13.],
       [ 21.,  18.,   2., ...,  63.,  64.,  50.],
       [ 21.,  -2.,  -2., ..., -41., -81., -33.]])

# Preprocessing

In [12]:
# normalization
# before normalize reshape data back to raw data shape
train_feature_2d = train_feature.reshape([-1, no_feature])
test_feature_2d = test_feature.reshape([-1, no_feature])

# min-max normalization
scaler3 = MinMaxScaler().fit(train_feature)
train_fea_norm1 = scaler3.transform(train_feature)
test_fea_norm1 = scaler3.transform(test_feature)
print('After normalization, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter normalization, the shape of test feature:', test_fea_norm1.shape)

# after normalization, reshape data to 3d in order to feed in to LSTM
train_fea_norm1 = train_fea_norm1.reshape([-1, segment_length, no_feature])
test_fea_norm1 = test_fea_norm1.reshape([-1, segment_length, no_feature])
print('After reshape, the shape of training feature:', train_fea_norm1.shape,
      '\nAfter reshape, the shape of test feature:', test_fea_norm1.shape)

After normalization, the shape of training feature: (129759, 192) 
After normalization, the shape of test feature: (43254, 192)
After reshape, the shape of training feature: (129759, 3, 64) 
After reshape, the shape of test feature: (43254, 3, 64)


In [13]:
BATCH_size = train_fea_norm1.shape[0] # use test_data as batch size

# feed data into dataloader
train_fea_norm1 = torch.tensor(train_fea_norm1).type('torch.FloatTensor')
train_label = torch.tensor(train_label.flatten())
train_data = Data.TensorDataset(train_fea_norm1, train_label)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_size, shuffle=False)

test_fea_norm1 = torch.tensor(test_fea_norm1).type('torch.FloatTensor')
test_label = torch.tensor(test_label.flatten())

## Feature Extraction

In [14]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(no_feature*segment_length, 64*4),
        )
        self.decoder = nn.Sequential(
            nn.Linear(64*4, no_feature*segment_length),
            nn.Sigmoid(),
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

autoencoder = AutoEncoder()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR)
loss_func = nn.MSELoss()

best_acc = []

## Classifiers - (Autoencoder)

In [18]:
n_class = int(11-len(removed_label))  # 0~9 classes ('10:rest' is not considered)
no_feature = 64  # the number of the features
segment_length = 480  # selected time window; 16=160*0.1
LR = 0.005  # learning rate
EPOCH = 20
n_hidden = 128  # number of neurons in hidden layer
l2 = 0.001  # the coefficient of l2-norm regularization

### LSTM

In [19]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()

        self.lstm_layer = nn.LSTM(
            input_size=no_feature,
            hidden_size=n_hidden,         # LSTM hidden unit
            num_layers=2,           # number of LSTM layer
            bias=True,
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, segment_length, no_feature)
        )

        self.out = nn.Linear(n_hidden, n_class)

    def forward(self, x):
        r_out, (h_n, h_c) = self.lstm_layer(x.float(), None)
        r_out = F.dropout(r_out, 0.3)

        test_output = self.out(r_out[:, -1, :]) # choose r_out at the last time step
        return test_output

lstm = LSTM()
print(lstm)

optimizer = torch.optim.Adam(lstm.parameters(), lr=LR, weight_decay=l2)   # optimize all parameters
loss_func = nn.CrossEntropyLoss()

LSTM(
  (lstm_layer): LSTM(64, 128, num_layers=2, batch_first=True)
  (out): Linear(in_features=128, out_features=11, bias=True)
)


In [20]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, accuracy_score
import time

# Ensure that numpy is imported at the top or before using it in the one_hot function
def one_hot(y_):
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_) + 1)  # Explicitly cast to int
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]


best_acc = []
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):
        output = lstm(train_x)  # LSTM output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

    if epoch % 10 == 0:
        test_output = lstm(test_fea_norm1)  # LSTM output of test data
        test_loss = loss_func(test_output, test_label.long())

        test_y_score = one_hot(test_label.data.cpu().numpy())  # .cpu() can be removed if your device is cpu.
        pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
        auc_score = roc_auc_score(test_y_score, pred_score)

        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
        pred_train = torch.max(output, 1)[1].data.cpu().numpy()

        test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
        train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)

        print('Epoch: ', epoch, '|train loss: %.4f' % loss.item(),
              ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
              'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
        best_acc.append(test_acc)
        best_auc.append(auc_score)


Epoch:  0 |train loss: 2.3791  train ACC: 0.2159 | test loss: 2.2634 test ACC: 0.4631 | AUC: 0.5017
Epoch:  10 |train loss: 1.9680  train ACC: 0.4652 | test loss: 1.9529 test ACC: 0.4631 | AUC: 0.5003


In [21]:
current_time = time.perf_counter()
running_time = current_time - start_time
print(classification_report(test_label.data.cpu().numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1723
         1.0       0.00      0.00      0.00      1669
         2.0       0.00      0.00      0.00      2569
         3.0       0.00      0.00      0.00      2393
         4.0       0.00      0.00      0.00      2556
         5.0       0.00      0.00      0.00      2451
         6.0       0.00      0.00      0.00      2547
         7.0       0.00      0.00      0.00      2348
         8.0       0.00      0.00      0.00      2277
         9.0       0.00      0.00      0.00      2688
        10.0       0.46      1.00      0.63     20033

    accuracy                           0.46     43254
   macro avg       0.04      0.09      0.06     43254
weighted avg       0.21      0.46      0.29     43254

BEST TEST ACC: 0.46314791695565727, AUC: 0.501723568892474
Total Running Time: 298.64 seconds


c:\Users\Sam\miniconda3\envs\pytgpu\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sam\miniconda3\envs\pytgpu\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Sam\miniconda3\envs\pytgpu\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### CNN

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # First 1D convolutional layer sequence
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=3,  # As your data has 3 features/channels per time step
                out_channels=16,
                kernel_size=4,
                stride=1,
                padding=2  # Adjust padding to control the output size
            ),
            nn.ReLU(),
            nn.MaxPool1d(4)  # Pooling to reduce the sequence length
        )
        # Second 1D convolutional layer sequence
        self.conv2 = nn.Sequential(
            nn.Conv1d(16, 32, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2)  # Further reduce the sequence length
        )
        # Adjust the following numbers based on the output size from conv2
        self.fc = nn.Linear(32 * 8, 128)  # You will need to calculate the correct size here
        self.out = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layer

        x = F.relu(self.fc(x))
        x = F.dropout(x, 0.2)  # Apply dropout

        output = self.out(x)
        return output, x  # Returns both the final output and the feature vector before the output layer

# Create an instance of CNN
cnn = CNN()
print(cnn)


CNN(
  (conv1): Sequential(
    (0): Conv1d(3, 16, kernel_size=(4,), stride=(1,), padding=(2,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(2,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=256, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


In [47]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR, weight_decay=l2)
loss_func = nn.CrossEntropyLoss()

best_acc = []
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):

        output = cnn(train_x)[0]  # CNN output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

    if epoch % 10 == 0:
        test_output = cnn(test_fea_norm1)[0]  # CNN output of test data
        test_loss = loss_func(test_output, test_label.long())

        test_y_score = one_hot(test_label.data.cpu().numpy())  # .cpu() can be removed if your device is cpu.
        pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
        auc_score = roc_auc_score(test_y_score, pred_score)

        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
        pred_train = torch.max(output, 1)[1].data.cpu().numpy()

        test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
        train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)


        print('Epoch: ', epoch,  '|train loss: %.4f' % loss.item(),
              ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
              'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
        best_acc.append(test_acc)
        best_auc.append(auc_score)

Epoch:  0 |train loss: 0.6932  train ACC: 0.4980 | test loss: 0.6937 test ACC: 0.4985 | AUC: 0.5354
Epoch:  10 |train loss: 0.6909  train ACC: 0.5133 | test loss: 0.6908 test ACC: 0.5387 | AUC: 0.6295
Epoch:  20 |train loss: 0.6664  train ACC: 0.6995 | test loss: 0.6609 test ACC: 0.6426 | AUC: 0.7683
Epoch:  30 |train loss: 0.5859  train ACC: 0.7073 | test loss: 0.5749 test ACC: 0.7228 | AUC: 0.7915
Epoch:  40 |train loss: 0.5234  train ACC: 0.7319 | test loss: 0.5271 test ACC: 0.7412 | AUC: 0.8177
Epoch:  50 |train loss: 0.4948  train ACC: 0.7584 | test loss: 0.4978 test ACC: 0.7532 | AUC: 0.8322
Epoch:  60 |train loss: 0.4637  train ACC: 0.7770 | test loss: 0.4710 test ACC: 0.7772 | AUC: 0.8531
Epoch:  70 |train loss: 0.4531  train ACC: 0.7867 | test loss: 0.4675 test ACC: 0.7741 | AUC: 0.8667
Epoch:  80 |train loss: 0.4324  train ACC: 0.8012 | test loss: 0.4365 test ACC: 0.8024 | AUC: 0.8772
Epoch:  90 |train loss: 0.4183  train ACC: 0.8091 | test loss: 0.4293 test ACC: 0.8082 | AUC

In [48]:
current_time = time.perf_counter()
running_time = current_time - start_time
print(classification_report(test_label.data.cpu().numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))

              precision    recall  f1-score   support

         0.0       0.84      0.72      0.78      1622
         1.0       0.76      0.87      0.81      1632

    accuracy                           0.80      3254
   macro avg       0.80      0.80      0.79      3254
weighted avg       0.80      0.80      0.79      3254

BEST TEST ACC: 0.8082360172095882, AUC: 0.8883928436510238
Total Running Time: 66.26 seconds


## GRU

In [73]:
# classifier
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()

        self.gru_layer = nn.GRU(
            input_size=no_feature,
            hidden_size=n_hidden,
            num_layers=2,
            bias=True,
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, segment_length, no_feature)
        )

        self.out = nn.Linear(n_hidden, n_class)

    def forward(self, x):
        r_out, (h_n, h_c) = self.gru_layer(x.float(), None)
        r_out = F.dropout(r_out, 0.3)
        test_output = self.out(r_out[:, -1, :]) # choose r_out at the last time step
        return test_output

gru = GRU()
print(gru)

optimizer = torch.optim.Adam(gru.parameters(), lr=LR, weight_decay=l2)   # optimize all parameters
loss_func = nn.CrossEntropyLoss()


GRU(
  (gru_layer): GRU(64, 128, num_layers=2, batch_first=True)
  (out): Linear(in_features=128, out_features=2, bias=True)
)


In [74]:
best_auc = []

# training and testing
start_time = time.perf_counter()
for epoch in range(EPOCH):
    for step, (train_x, train_y) in enumerate(train_loader):

        output = gru(train_x)  # GRU output of training data
        loss = loss_func(output, train_y.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients

    if epoch % 10 == 0:
        test_output = gru(test_fea_norm1)  # GRU output of test data
        test_loss = loss_func(test_output, test_label.long())

        test_y_score = one_hot(test_label.data.cpu().numpy())
        pred_score = F.softmax(test_output, dim=1).data.cpu().numpy()  # normalize the output
        auc_score = roc_auc_score(test_y_score, pred_score)

        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()
        pred_train = torch.max(output, 1)[1].data.cpu().numpy()

        test_acc = accuracy_score(test_label.data.cpu().numpy(), pred_y)
        train_acc = accuracy_score(train_y.data.cpu().numpy(), pred_train)

        print('Epoch: ', epoch, '|train loss: %.4f' % loss.data.item(),
              ' train ACC: %.4f' % train_acc, '| test loss: %.4f' % test_loss.item(),
              'test ACC: %.4f' % test_acc, '| AUC: %.4f' % auc_score)
        best_acc.append(test_acc)
        best_auc.append(auc_score)
current_time = time.perf_counter()
running_time = current_time - start_time

print(classification_report(test_label.data.numpy(), pred_y))
print('BEST TEST ACC: {}, AUC: {}'.format(max(best_acc), max(best_auc)))
print("Total Running Time: {} seconds".format(round(running_time, 2)))

Epoch:  0 |train loss: 0.6966  train ACC: 0.4997 | test loss: 0.7995 test ACC: 0.4991 | AUC: 0.4959
Epoch:  10 |train loss: 0.6951  train ACC: 0.4996 | test loss: 0.6969 test ACC: 0.5009 | AUC: 0.4975
Epoch:  20 |train loss: 0.6932  train ACC: 0.4975 | test loss: 0.6928 test ACC: 0.5061 | AUC: 0.5185
Epoch:  30 |train loss: 0.6932  train ACC: 0.4983 | test loss: 0.6934 test ACC: 0.5052 | AUC: 0.5031
Epoch:  40 |train loss: 0.6930  train ACC: 0.5067 | test loss: 0.6931 test ACC: 0.4969 | AUC: 0.5024
Epoch:  50 |train loss: 0.6930  train ACC: 0.5036 | test loss: 0.6929 test ACC: 0.5129 | AUC: 0.5196
Epoch:  60 |train loss: 0.6930  train ACC: 0.5075 | test loss: 0.6930 test ACC: 0.5098 | AUC: 0.5103
Epoch:  70 |train loss: 0.6930  train ACC: 0.5130 | test loss: 0.6931 test ACC: 0.5071 | AUC: 0.5075
Epoch:  80 |train loss: 0.6929  train ACC: 0.5140 | test loss: 0.6930 test ACC: 0.5108 | AUC: 0.5136
Epoch:  90 |train loss: 0.6928  train ACC: 0.5137 | test loss: 0.6930 test ACC: 0.5123 | AUC

## Classifier - CSP